## 使用 COPT 求解线性分式规划（单变量版本）

本笔记展示了如何求解一个**线性分式规划（LFP）**问题，  
该问题仅包含**一个决策变量**，便于理解和学习。

### 原始问题形式

我们要解决的问题是：

$$
\min_x \quad \frac{c \cdot x + d}{e \cdot x + f}
$$

约束条件为：

$$
a \cdot x \le b, \quad e \cdot x + f > 0
$$

---

### 变量替换

为了将分式目标函数线性化，引入以下变量：

- $z = \frac{1}{e \cdot x + f}$
- $y = \frac{x}{e \cdot x + f} = x \cdot z$
- 所以，$x = \frac{y}{z}$

---

### 转化后的线性规划问题

现在我们可以求解以下问题：

$$
\min_{y, z} \quad c \cdot y + d \cdot z
$$

满足以下约束条件：

$$
a \cdot y - b \cdot z \le 0
$$

$$
e \cdot y + f \cdot z = 1
$$

$$
z \ge 0
$$

---

接下来，我们将使用 **COPT Python 接口**来实现这个模型。


### 第 1 步：导入并初始化 COPT 求解器环境

In [ ]:
from coptpy import *
env = Envr()
model = env.createModel("FractionalLP")

### 第 2 步：定义模型中的数据参数

In [ ]:
# Step 2: Define your data
a = 2     
b = 6     
c = -3    
d = 1     
e = 1    
f = 2

### 第 3 步：添加决策变量

In [ ]:
# Step 3: Add decision variables
# y 可以是负数, z 必须为非负
y = model.addVar(lb=-COPT.INFINITY, name="y")
z = model.addVar(lb=0.0, name="z")

### 第 4 步：设置目标函数

In [ ]:
# Step 4: Set the objective function: minimize c * y + d * z
model.setObjective(c * y + d * z, sense=COPT.MINIMIZE)

### 第 5 步：添加约束条件

In [ ]:
# Step 5: Add constraints
# Constraint: a * y - b * z <= 0
model.addConstr(a * y - b * z <= 0, name="ineq")
# Constraint: e * y + f * z = 1
model.addConstr(e * y + f * z == 1, name="normalization")

### 第 6 步：求解

In [ ]:
# Step 6: Solve the model
model.solve()

### 第 7 步：打印结果

In [ ]:
# Step 7: Print results
if model.status == COPT.OPTIMAL:
    print("Optimal objective value:", model.objVal)
    print("Optimal y value:", y.x)
    print("Optimal z value:", z.x)

    # Recover original x from y and z
    x = y.x / z.x
    print("Recovered x value (original variable):", x)
else:
    print("Solver status:", model.status)
    print("Solution is not available")

### 求解器运行结果说明（结果解析）
下面是 COPT 求解器输出的结果说明：

---
#### 最优目标函数值： 
Optimal objective value: -1.5999999999999999

这是线性化后的目标函数的最小值，对应：

$$
\min \; c \cdot y + d \cdot z = -3 \cdot y + 1 \cdot z
$$

结果约为 $-1.6$，是合理且精确的结果。

---

这是线性化模型中变量 $y$ 和 $z$ 的最优解：

- $y = \frac{x}{e x + f}$
- $z = \frac{1}{e x + f}$

---

#### 恢复原始变量 $x$
Recovered x value (original variable): 2.9999999999999996



根据变量替换关系 $x = \frac{y}{z}$，我们可以得到原始变量的值：

$$
x = \frac{0.6}{0.2} = 3.0
$$

---

#### 验证原始目标函数值

将 $x = 3$ 带入原始目标函数进行验证：

$$
\frac{c \cdot x + d}{e \cdot x + f} = \frac{-3 \cdot 3 + 1}{1 \cdot 3 + 2} = \frac{-8}{5} = -1.6
$$

完全匹配求解器的输出结果。

---

#### 总结表格

| 变量        | 值       |
|-------------|----------|
| 最优 $x$    | 3.0      |
| 目标值      | -1.6     |
| $y = x z$   | 0.6      |
| $z = 1 / (e x + f)$ | 0.2 |

以上结果验证了线性分式问题的建模和求解过程是正确且稳定的。
